In [4]:
import numpy as np
import os
import pandas as pd
from collections import Counter

In [127]:
def load_data(fPath):
    df = pd.read_csv(fPath, header = None, skipinitialspace = True)
    # get classes to list
    data = df.iloc[:,:].to_numpy().copy()
    # get data to list
    #print(data.groupby(by=0).count())
    #print(df)
    #df2 = df.groupby([4,0], as_index = False).count()
    #print(df2)
    
    #df1 = df.iloc[:, 0:-1].to_numpy()
    #print(df1)
    return data

def count_values_from_frame(data):
    # get columns of frame
    columns = list(data)
    featureCounts = dict()
    # Count appearences of elements in all columns
    for c in columns:
        featureCounts[c] = df[c].value_counts()
    #print(featureCounts)
    return

def count_features_per_class(data, normalize = True):
    classes = np.unique(data[:,-1])
    #print(data)
    #counter=collections.Counter(a)
    # Create a list of dictionaries, one for each feature. Each dictionary contains an entry for each unique feature value(key)
    # and a list that contains the freq count for each class.
    #attributes = [[dict( ((key for key in a[0]), [0 for c in range(0, len(classes))]) for a in zip(np.unique(data[:,f], return_counts = True)))] for f in range(data.shape[1]-1)]
    #attributes = [[dict( (a, [0 for c in range(0, len(classes))]) for a in np.unique(data[:,f]))] for f in range(data.shape[1]-1)]
    #print(attributes)
    counterDict = []#[{} for x in range(data.shape[1]-1)]
    #
    classStats = np.zeros(len(classes))
    for j, c in enumerate(classes):
        for f in range(data.shape[1]-1):           
            allKeys = np.unique(data[:,f])
            #print(allKeys)
            if len(counterDict) < f+1:
                counterDict.append(dict( (a, [0 for c in range(0, len(classes))]) for a in allKeys))
            
            keys, freqs = (np.unique(data[[data[:,-1] == c]][:,f], return_counts = True))
            for i,k in enumerate(keys):
                counterDict[f][k][j] = freqs[i] 
        # Compute per class frequencies, for comptueting info gain
        for f in range(data.shape[0]):
            if data[f,-1] == c:
                classStats[j] += 1
   
    return counterDict, classStats/data.shape[0]
    
def comp_feat_entropy(dataDict, epsilon = 0.001):
    """
        ARGUMENTS: dataDict (list of dicts): A list holding one dictionary per feature. Each discionary holds an entry with a kew for each unique variable.
                                             Each variable's (key's) value is a list containing the accurencies each variable appears in each class. So the struct
                             is: numOfFeatures Dicts of: numOfVariables in Feature Keys with: numOfclasses occurencies.
    """
    total = 0
    #print(dataDict[0])
    # Iterate over features
    for i, f in enumerate(dataDict):
        a = np.array([f[k] for k in f.keys()])
        total += np.sum(a)
        #print(a, a.sum())
        featFracs  = [np.sum(f)/total for f in a]
        classFracs = np.array([f/np.sum(f) for f in a])
        # Epsilon guards against hte log2(0) case
        classEntropy = np.array([-np.sum(((f[:]+epsilon) * np.log2(f[:]+epsilon))) for f in classFracs])
        featEntropy = featFracs * classEntropy
        #print(featFracs, classFracs, classEntropy)
        #print("\n", featEntropy)
        return featEntropy
        
def compute_feat_info_gain(dataDict, classDist, epsilon = 0.0001):
    featEntropy = comp_feat_entropy(dataDict)    
    classEntropy = -np.sum((classDist[:]) * np.log2(classDist[:]))
    infoGain = classEntropy - featEntropy
    #print(infoGain)
    return infoGain
    
def compute_attribure_entropy():
    a =0 


In [91]:
fPath = "dummy2.txt"
data = load_data(fPath)
print(data.shape)
print(data)
print(data[data[:,-1] == 'Yes'])

(6, 5)
[['a' 'b' 'c' 'd' 'No']
 ['c' 'f' 'g' 'd' 'No']
 ['a' 'b' 'c' 'd' 'Yes']
 ['z' 'w' 'e' 'e' 'No']
 ['a' 'c' 'e' 'f' 'No']
 ['v' 'b' 'n' 'f' 'Yes']]
[['a' 'b' 'c' 'd' 'Yes']
 ['v' 'b' 'n' 'f' 'Yes']]


In [97]:
dataDict, classDist = count_features_per_class(data)

<ipython-input-96-cc2a52ab3677>:44: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keys, freqs = (np.unique(data[[data[:,-1] == c]][:,f], return_counts = True))


In [101]:
featEntropy = comp_feat_entropy(dataDict)

In [149]:
featureInfoGain = compute_feat_info_gain(dataDict, classDist)
order = np.argsort(featureInfoGain)
print(featureInfoGain)
print("Most Informative feature: ", order[-1], " with Info gain: ", featureInfoGain[-1])
print("Complete order:",*["Feature {}, Info Gain: {}".format(order[-i], featureInfoGain[-i]) for i in order[::-1]],sep = '\n')

[0.45950727 0.91687544 0.91687544 0.91687544]
Most Informative feature:  3  with Info gain:  0.9168754393650592
Complete order:
Feature 1, Info Gain: 0.9168754393650592
Feature 2, Info Gain: 0.9168754393650592
Feature 3, Info Gain: 0.9168754393650592
Feature 0, Info Gain: 0.4595072712486298
